In [20]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
from selenium import webdriver
from datetime import date
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # полезно чтобы избежать ругательства при создании нового поля таблицы(почемуто все-время ругня)

In [21]:
# у нас должны быть файлы со списками исключений тех ссылок, которые уже просмотрены
# function returns list of links 
def open_seen(file):
    with open(file,'r') as f:
        content=f.read()
        seen=eval(content)
    return seen

In [22]:
# если нужно записать файл из листа ссылок
def write_seen(file,list_links):
    with open(file, 'w') as f:
        print(list_links, file=f)

In [23]:
# в гугле собирает ссылки по запросу,работает только по заданному количеству листов
def google_q (query,n_pages):
    # query example:'site:rabota.ua/cv/ AND "коммерческий директор" AND "Украина" AND "FMCG"'
    # один раз устанавливаем драйвер https://sites.google.com/a/chromium.org/chromedriver/downloads

    # загружаем драйвер, открывается новое окно хрома, в этом окне делаем запрос на поиск, с каждой страницы
    # собираем нужные ссылки, закрываем драйвер

    
    # google search 
    # start driver (new chrome window)
    driver = webdriver.Chrome('C:\\ProgramData\chromedriver.exe')
    driver.get('https:www.google.com')
    time.sleep(3)

    # запрос на поиск 
    search_query = driver.find_element_by_name('q')
    search_query.send_keys(query)
    time.sleep(0.5)

    # имитация ввода запроса на поиск
    search_query.send_keys(u'\ue007')
    time.sleep(3)

    # формируем список ссылок с первых десяти страниц
    list_link=[]
    for i in range(n_pages):
        list_link.extend(ele(driver))
        next_page=driver.find_element_by_xpath("//*[contains(local-name(),'span') and contains(text(),'Уперед')]")
        next_page.click()
        time.sleep(5)
    driver.quit()
    return list_link

In [24]:
# вспомагательная для google_q только для ссылок с паттерном 
def ele(driver):
# получение всех ссылок rabota.ua на одной странице
    elems = driver.find_elements_by_xpath("//a[@href]")
    elems= [elem.get_attribute("href")for elem in elems]
    ln_links = [x for x in elems if re.search(r'^https://rabota.ua/cv', x)]
    return ln_links

In [25]:
# вспомагательная для получения супа, один раз получаем, потом из него вынимаем то, что нужно
def get_soup(link):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    return soup

In [26]:
# вынимаем из супа только единичные значения для одной ссылки и формируем таблицу
def soup_potato(link,soup):
    price=None
    brief=None
    age=None
    city=None
    exp=None
    edu=None
    lang=None
          
    
    
    refresh_year=soup.find('span',{'class':'muted'}).text
    update_year=re.findall('\d+',refresh_year)[1]
    
    name=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblName').text
    
    short=soup.find('p',{'class':'rua-p-t_12'}).text.strip().split('\r\n')
    city=short[0]
    
    regex = re.compile('\d+')
    age=regex.findall(short[1])[0]
    
    try:
        regex = re.compile('\d+')
        price=regex.findall(short[2])[0]+regex.findall(short[2])[1]
    except:
        pass
    try:
        brief=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_Skills_txtSkills').text.replace('Программное обеспечение и сервисы: ','')
    except:
        pass
    exp=soup.find('div',id='ExperienceHolder').text.replace('Опыт работы','').replace('Нет опыта работы','')
    edu=soup.find('div',id='EducationHolder').text.replace('Образование','')
    lang=soup.find('div',id='LanguagesHolder').text.replace('Владение языками','')
    scrap_date=date.today()
    field1={'name':name,'link':link,'update_year':update_year,'city':city,'age':age,'price':price,'brief':brief,'exp':exp,'edu':edu,'lang':lang,'scrap_date':scrap_date}
    df=pd.DataFrame([field1])
    
    return df

In [27]:
# вынимаем из супа множественные значения из каждой ссылки, например, история работы
def soup_detail(link,soup):
    gde=[]
    kogda=[]
    b1=soup.find('div',id='ExperienceHolder').find_all('b')
    m=soup.find('div',id='ExperienceHolder').find_all('em')
    for i in b1:
        gde.append(i.text.strip())
    for k in m:
        kogda.append(k.text.strip())
    
    # если слово выделено, то оно попадает в gde, там образуется шлак
    # попыталась сделать ручные исключения
    # работает плохо, но как попытка
    
    gde=[x for x in gde if "Достиж" not in x]
    gde=[x for x in gde if "Рекоменд" not in x]
    gde=[x for x in gde if "улучш" not in x]
    gde=[x for x in gde if "Результ" not in x]
    gde=[x for x in gde if "Обязанн" not in x]
    gde=[x for x in gde if "достиж" not in x]
    gde=gde[:min(len(gde),len(kogda))] # привожу два листа к одной размерности
    
    kogda=kogda[:min(len(gde),len(kogda))]# привожу два листа к одной размерности
    d = {'должность':gde,'период':kogda} # создаю словать и dataframe
    uh=pd.DataFrame(d)
    
    uh['предприятие']=uh['должность'].shift(-1)# копирую и поднимаю столбец, чтобы сделать строки
    uh['отрасль']=uh['период'].shift(-1)# копирую и поднимаю столбец, чтобы сделать строки
    
    
    
    uhh=uh.iloc[::2, :] # беру каждую вторую строку
    
    # извлекаю из строчного периода цифровой год и месяц в duration
    regex=re.compile('(\(\d+.лет|\(\d+.год)')
    uhh['год']=uhh['период'].str.extract(regex)
    uhh['год']=uhh['год'].str.extract(r'(\d+)').fillna(0)
    uhh['мес']=uhh['период'].str.extract('(\d+).мес').fillna(0)
    uhh['мес1']=uhh['мес'].astype(int)/12
    uhh['duration']=uhh['год'].astype(int)+uhh['мес1'].round(2)
    
    #формирую таблицу
    uhh['link']=link
    uhh=uhh[['link','предприятие','должность','период','отрасль','duration']].reset_index(drop=True)
    return uhh

In [28]:
# перемешиваем все данные по всем ссылкам в одну таблицу
def soup_stir(links):
    ddf2=pd.DataFrame(columns=['link', 'предприятие', 'должность', 'период', 'отрасль', 'duration',
       'age', 'brief', 'city', 'edu', 'exp', 'lang', 'name', 'price',
       'scrap_date', 'update_year'])
    for i in links:
        soup=get_soup(i)
        df1= soup_potato(i,soup)
        df2=soup_detail(i,soup)
        ddf1=df1.merge(df2)
        df_s1=pd.concat([ddf2,ddf1])
        ddf2=df_s1.copy()
    return df_s1

In [29]:
# делаем красивый консолидированный пивот
def pivot_from_soup_stir(d):
    d=d.fillna(' ')
    d1=d.reset_index()
    d2=d1.pivot_table(index=['link','name','age','brief','update_year','предприятие','должность','отрасль','index'],values='duration')
    d3=d2.sort_index(level=['link','index'])
    return d3

In [30]:
all_seen=open_seen('links_seen/all_seen_till_061219.txt')

In [31]:
len(all_seen)

169

In [32]:
rabota_CCO=google_q('site:rabota.ua/cv/ AND "коммерческий директор" AND "Украина" AND "FMCG"',2)

In [33]:
len(rabota_CCO) #количество ссылок по запросу

19

In [34]:
len([x for x in rabota_CCO if x not in all_seen]) #количество ссылок, учитывая исключения

2

In [35]:
links1=[x for x in rabota_CCO if x not in all_seen]
len(np.unique(links1)) #проверка на уникальность

2

In [36]:
d=soup_stir(links1[:5])

In [37]:
d

,age,brief,city,duration,edu,exp,lang,link,name,price,scrap_date,update_year,должность,отрасль,период,предприятие
0,46,Руководитель в области продаж с более 15 - лет...,Одесса,1.50,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Руководитель отдела развития,АПК (Агропромышленный комплекс),фев 2017 - июл 2018 (1 год 6 мес),ООО Велес Агро Брокер (Агроэкспорт)
1,46,Руководитель в области продаж с более 15 - лет...,Одесса,2.00,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Региональный менеджер,FMCG,фев 2015 - янв 2017 (2 года),ООО «Ритейл Профешинлз Украина»
2,46,Руководитель в области продаж с более 15 - лет...,Одесса,1.50,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Региональный менеджер по работе с ключевыми кл...,более 500 сотрудников\r\n\t\t\t\t\tFMCG,июл 2013 - дек 2014 (1 год 6 мес),Кока-Кола Беверіджиз Україна
3,46,Руководитель в области продаж с более 15 - лет...,Одесса,2.33,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Региональный Менеджер по работе с ключевыми кл...,FMCG,мар 2011 - июн 2013 (2 года 4 мес),ПАО Мироновский Хлебопродукт (ТМ Наша Ряба)
4,46,Руководитель в области продаж с более 15 - лет...,Одесса,3.08,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Региональный менеджер по работе с ключевыми к...,FMCG,янв 2008 - янв 2011 (3 года 1 мес),ЗАО Чумак
5,46,Руководитель в области продаж с более 15 - лет...,Одесса,5.17,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Региональный менеджер по работе с ключевыми кл...,более 500 сотрудников\r\n\t\t\t\t\tFMCG,авг 2002 - сен 2007 (5 лет 2 мес),"Монделіс Україна, ПАТ / Крафт Фудз Україна"
6,46,Руководитель в области продаж с более 15 - лет...,Одесса,4.08,Одесский Государственный Политехнический Униве...,\r\nРуководитель отдела развитияфев 2017 - июл...,Английский- базовый\r\n\t\t\t\r\n\t\tУкраински...,https://rabota.ua/cv/9778845,Олег Николаевич,None,2019-12-17,2019,Супервайзер,более 500 сотрудников\r\n\t\t\t\t\tТорговля оп...,янв 1998 - янв 2002 (4 года 1 мес),"АЛМИ-Трейд, ООО"
0,44,None,Киев,5.58,Переяслав-Хмельницкий государственный педагоги...,\r\nРегиональный менеджер FMCGиюн 2014 - насто...,Русский- родной\r\n\t\t\t\r\n\t\tУкраинский- р...,https://rabota.ua/cv/4760938,Александр,None,2019-12-17,2019,Региональный менеджер FMCG,от 100 до 250 сотрудников\r\n\t\t\t\t\tFMCG,июн 2014 - настоящее время (5 лет 7 мес),"С-транс, ООО / Inmilkco"
1,44,None,Киев,1.50,Переяслав-Хмельницкий государственный педагоги...,\r\nРегиональный менеджер FMCGиюн 2014 - насто...,Русский- родной\r\n\t\t\t\r\n\t\tУкраинский- р...,https://rabota.ua/cv/4760938,Александр,None,2019-12-17,2019,Коммерческий директор,от 20 до 50 сотрудников\r\n\t\t\t\t\tТорговля ...,янв 2013 - июн 2014 (1 год 6 мес),Геотранс
2,44,None,Киев,4.25,Переяслав-Хмельницкий государственный педагоги...,\r\nРегиональный менеджер FMCGиюн 2014 - насто...,Русский- родной\r\n\t\t\t\r\n\t\tУкраинский- р...,https://rabota.ua/cv/4760938,Александр,None,2019-12-17,2019,руководитель отдела ВЭД,FMCG,окт 2008 - дек 2012 (4 года 3 мес),ДП Милкил

In [38]:
pivot_from_soup_stir(d)

duration
link                         name            age brief                                              update_year предприятие                                 должность                                          отрасль                                            index          
https://rabota.ua/cv/4760938 Александр       44                                                     2019        С-транс, ООО / Inmilkco                     Региональный менеджер FMCG                         от 100 до 250 сотрудников\r\n\t\t\t\t\tFMCG        0          5.58
                                                                                                                Геотранс                                    Коммерческий директор                              от 20 до 50 сотрудников\r\n\t\t\t\t\tТорговля о... 1          1.50
                                                                                                                ДП Милкиленд-Украина                        руководитель отдела ВЭД                            FMCG                                               2          4.25
                                                                                                                менеджер ВЭД                                Директор ВЭД                                       от 20 до 50 сотрудников\r\n\t\t\t\t\tFMCG          3          1.83
                                                                                                                Торговый дом Любас                          Менеджер по работе с торговыми сетями              от 50 до 100 сотрудников\r\n\t\t\t\t\tТорговля ... 4          3.33
https://rabota.ua/cv/9778845 Олег Николаевич 46  Руководитель в области продаж с более 15 - летн... 2019        ООО Велес Агро Брокер (Агроэкспорт)         Руководитель отдела развития                       АПК (Агропромышленный комплекс)                    0          1.50
                                                                                                                ООО «Ритейл Профешинлз Украина»             Региональный менеджер                              FMCG                                               1          2.00
                                                                                                                Кока-Кола Беверіджиз Україна                Региональный менеджер по работе с ключевыми кли... более 500 сотрудников\r\n\t\t\t\t\tFMCG            2          1.50
                                                                                                                ПАО Мироновский Хлебопродукт (ТМ Наша Ряба) Региональный Менеджер по работе с ключевыми кли... FMCG                                               3          2.33
                                                                                                                ЗАО Чумак                                   Региональный  менеджер по работе с ключевыми кл... FMCG                                               4          3.08
                                                                                                                Монделіс Україна, ПАТ / Крафт Фудз Україна  Региональный менеджер по работе с ключевыми кли... более 500 сотрудников\r\n\t\t\t\t\tFMCG            5          5.17
                                                                                                                АЛМИ-Трейд, ООО                             Супервайзер                                        более 500 сотрудников\r\n\t\t\t\t\tТорговля опт... 6          4.08